token : q3D+yarl7RWbNx9w5GccNE405MY3PQLeW9mL52roBDupSK0Q7ZWUdly21XU2PsdTOXrEXf7GnPHyGyz5bIzsM4R7zbvvuG8GOyz69FUedB/Wyn2X+EtsMe4K9SUnU/Nf9Eoz3vfDxzXZZrXaimUWLAdB04t89/1O/w1cDnyilFU=

secret : dc77d38f2b719db16ccff3434f674945

Your Authtoken(ngrok) : 2oXpvC6DlHXfz2L25pJoJTLFDpe_3CfL75t2pgmmFjbQamRhn


# **安裝必要的套件**

In [ ]:
!pip install line-bot-sdk
!pip install flask
!pip install pyngrok

# **找到網址**

In [5]:
from pyngrok import ngrok
# 使用您的 authtoken 進行驗證
!ngrok config add-authtoken 2oXpvC6DlHXfz2L25pJoJTLFDpe_3CfL75t2pgmmFjbQamRhn
# 繼續啟動 ngrok 和 Flask 應用程式
public_url = ngrok.connect(5000)
print("Public URL:", public_url)

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
Public URL: NgrokTunnel: "https://35d9-35-194-184-15.ngrok-free.app" -> "http://localhost:5000"


# **認識否連線**

In [ ]:
from flask import Flask, request
from pyngrok import ngrok
import json

app = Flask(__name__)

@app.route("/", methods=['POST'])
def linebot():
    body = request.get_data(as_text=True)
    print("Received:", body)
    return 'OK'

# 啟動 Flask 應用程式在 5000 端口
app_port = 5000
ngrok_tunnel = ngrok.connect(app_port)
print("Public URL:", 'https://35d9-35-194-184-15.ngrok-free.app')  # 這是您可以用來作為 Webhook 的公開 URL
app.run(port=app_port)

# **測試**

base code (回覆相同的訊息)

In [ ]:
from flask import Flask, request, abort
from linebot import LineBotApi, WebhookHandler
from linebot.exceptions import InvalidSignatureError
from linebot.models import MessageEvent, TextMessage, TextSendMessage
from pyngrok import ngrok

# 建立 Flask 應用程式
app = Flask(__name__)

# 設置你的 Channel Access Token 和 Channel Secret
CHANNEL_ACCESS_TOKEN = '你的 Channel Access Token'
CHANNEL_SECRET = '你的 Channel Secret'
line_bot_api = LineBotApi(CHANNEL_ACCESS_TOKEN)
handler = WebhookHandler(CHANNEL_SECRET)

# 設置 Webhook 處理函數
@app.route("/callback", methods=['POST'])
def callback():
    # 獲取 HTTP 請求中的 X-Line-Signature 標頭
    signature = request.headers['X-Line-Signature']
    # 將請求內容轉為文本
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # 驗證請求的合法性
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return 'OK'

# 設置接收訊息的事件
@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    # 回覆相同的訊息
    message = TextSendMessage(text=event.message.text)
    line_bot_api.reply_message(event.reply_token, message)

if __name__ == "__main__":
    app.run(port=5000)

確認是否為文字(TextSendMessage發送純文字)

In [ ]:
import json
from flask import Flask, request, abort
from linebot import LineBotApi, WebhookHandler
from linebot.exceptions import InvalidSignatureError
from linebot.models import MessageEvent, TextMessage, TextSendMessage
from pyngrok import ngrok

app = Flask(__name__)

access_token = 'YOUR_CHANNEL_ACCESS_TOKEN'
secret = 'YOUR_CHANNEL_SECRET'
line_bot_api = LineBotApi(access_token)  # 使用新的 LineBotApi
handler = WebhookHandler(secret)         # 使用新的 WebhookHandler

@app.route("/", methods=['POST'])
def linebot():
    body = request.get_data(as_text=True)
    signature = request.headers.get('X-Line-Signature')

    if not signature:
        abort(400)  # 若沒有簽章，直接回傳 400 錯誤

    try:
        handler.handle(body, signature)  # 綁定訊息回傳的相關資訊
        json_data = json.loads(body)

        # 取得回傳訊息的 Token 和訊息類型
        tk = json_data['events'][0]['replyToken']
        msg_type = json_data['events'][0]['message']['type']

        # 回傳訊息的內容
        if msg_type == 'text':
            msg = json_data['events'][0]['message']['text']
            reply = f"收到訊息：{msg}"
        else:
            reply = "你傳的不是文字訊息喔～"

        # 回傳訊息
        line_bot_api.reply_message(tk, TextSendMessage(reply))
    except InvalidSignatureError:
        abort(400)  # 若簽章驗證失敗，回傳 400 錯誤
    except Exception as e:
        print(f"Error: {e}")
        abort(500)  # 若其他錯誤發生，回傳 500 錯誤

    return 'OK'

if __name__ == "__main__":
    app.run(port=5000)

回傳文字(確認user的訊息)

In [ ]:
import random
from flask import Flask, request, abort
from linebot import LineBotApi, WebhookHandler
from linebot.exceptions import InvalidSignatureError
from linebot.models import MessageEvent, TextMessage, TextSendMessage
from pyngrok import ngrok

app = Flask(__name__)

CHANNEL_ACCESS_TOKEN = '你的 Channel Access Token'
CHANNEL_SECRET = '你的 Channel Secret'

line_bot_api = LineBotApi(CHANNEL_ACCESS_TOKEN)
handler = WebhookHandler(CHANNEL_SECRET)

# 設置 Webhook 處理函數
@app.route("/callback", methods=['POST'])
def callback():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return 'OK'

# 設置接收訊息的事件
@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    user_message = event.message.text
    if user_message == "今天可以吃什麼":
        # 隨機選擇一種餐點
        options = ["漢堡", "三明治", "吐司", "蛋餅"]
        reply_message = random.choice(options)
        message = TextSendMessage(text=f"你可以吃 {reply_message}！")
    else:
        # 回覆相同的訊息
        message = TextSendMessage(text=user_message)

    line_bot_api.reply_message(event.reply_token, message)

if __name__ == "__main__":
    app.run(port=5000)

依照時間輸出訊息

In [ ]:
from datetime import datetime
import random
from flask import Flask, request, abort
from linebot import LineBotApi, WebhookHandler
from linebot.exceptions import InvalidSignatureError
from linebot.models import MessageEvent, TextMessage, TextSendMessage
from pyngrok import ngrok

# 建立 Flask 應用程式
app = Flask(__name__)

# 設置你的 Channel Access Token 和 Channel Secret
CHANNEL_ACCESS_TOKEN = '你的 Channel Access Token'
CHANNEL_SECRET = '你的 Channel Secret'

line_bot_api = LineBotApi(CHANNEL_ACCESS_TOKEN)
handler = WebhookHandler(CHANNEL_SECRET)

# 設置 Webhook 處理函數
@app.route("/callback", methods=['POST'])
def callback():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return 'OK'

# 設置接收訊息的事件
@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    user_message = event.message.text
    current_hour = datetime.now().hour

    # 根據不同的時間回覆相應的訊息
    if 6 <= current_hour < 12:
        reply_message = "早安！新的一天開始了，今天要做些什麼好呢？"
    elif 12 <= current_hour < 18:
        reply_message = "午安！要不要喝杯咖啡提神一下？"
    elif 18 <= current_hour < 24:
        reply_message = "晚安！一天快結束了，準備好好休息吧！"
    else:
        reply_message = "深夜了，該好好休息，晚安！"

    message = TextSendMessage(text=reply_message)
    line_bot_api.reply_message(event.reply_token, message)

if __name__ == "__main__":
    app.run(port=5000)

添加任務

In [ ]:
from flask import Flask, request, abort
from linebot import LineBotApi, WebhookHandler
from linebot.exceptions import InvalidSignatureError
from linebot.models import MessageEvent, TextMessage, TextSendMessage
from pyngrok import ngrok

# 建立 Flask 應用程式
app = Flask(__name__)

# 設置你的 Channel Access Token 和 Channel Secret
CHANNEL_ACCESS_TOKEN = '你的 Channel Access Token'
CHANNEL_SECRET = '你的 Channel Secret'

line_bot_api = LineBotApi(CHANNEL_ACCESS_TOKEN)
handler = WebhookHandler(CHANNEL_SECRET)

# 模擬的用戶狀態和任務列表（一般在多用戶情況下會用資料庫）
user_states = {}  # 儲存每位用戶的操作狀態
task_list = []  # 任務列表

# 設置 Webhook 處理函數
@app.route("/callback", methods=['POST'])
def callback():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return 'OK'

# 設置接收訊息的事件
@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    user_id = event.source.user_id
    user_message = event.message.text

    # 如果用戶訊息是「添加任務」
    if user_message == "添加任務":
        user_states[user_id] = "adding_task"
        task_str = "\n".join(task_list) if task_list else "目前沒有任務。"
        reply_message = f"現在的任務列表：\n{task_str}\n\n請輸入您想要添加的任務。"
        line_bot_api.reply_message(event.reply_token, TextSendMessage(text=reply_message))

    # 如果用戶在添加任務模式中
    elif user_states.get(user_id) == "adding_task":
        # 添加新任務到列表
        task_list.append(user_message)
        user_states[user_id] = None  # 清除狀態
        task_str = "\n".join(task_list)
        reply_message = f"任務已添加成功！\n\n目前的任務列表：\n{task_str}"
        line_bot_api.reply_message(event.reply_token, TextSendMessage(text=reply_message))

    else:
        # 回覆一般訊息
        line_bot_api.reply_message(event.reply_token, TextSendMessage(text="請輸入有效指令，例如「添加任務」。"))

if __name__ == "__main__":
    app.run(port=5000)

回復貼圖

In [ ]:
from flask import Flask, request, abort
from linebot import LineBotApi, WebhookHandler
from linebot.exceptions import InvalidSignatureError
from linebot.models import MessageEvent, TextMessage, TextSendMessage, StickerSendMessage
from pyngrok import ngrok

# 建立 Flask 應用程式
app = Flask(__name__)

# 設置你的 Channel Access Token 和 Channel Secret
CHANNEL_ACCESS_TOKEN = '你的 Channel Access Token'
CHANNEL_SECRET = '你的 Channel Secret'

line_bot_api = LineBotApi(CHANNEL_ACCESS_TOKEN)
handler = WebhookHandler(CHANNEL_SECRET)

# 設置 Webhook 處理函數
@app.route("/callback", methods=['POST'])
def callback():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return 'OK'

# 設置接收訊息的事件
@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    user_message = event.message.text

    if user_message == "傳送貼圖":
        # 傳送貼圖 (使用貼圖包 ID 1 和貼圖 ID 1)
        sticker_message = StickerSendMessage(
            package_id='1',
            sticker_id='1'
        )
        line_bot_api.reply_message(event.reply_token, sticker_message)

    elif user_message == "傳送文字和貼圖":
        # 傳送文字和貼圖
        text_message = TextSendMessage(text="這是一個貼圖")
        sticker_message = StickerSendMessage(
            package_id='1',
            sticker_id='2'
        )
        line_bot_api.reply_message(event.reply_token, [text_message, sticker_message])

    else:
        # 回覆一般訊息
        line_bot_api.reply_message(event.reply_token, TextSendMessage(text="請輸入有效指令，例如「傳送貼圖」。"))

if __name__ == "__main__":
    app.run(port=5000)

Buttons Template **可以顯示一張圖片、標題、描述文字和多個按鈕選項。**

In [ ]:
'''
from linebot.models import TemplateSendMessage, ButtonsTemplate, MessageAction, URIAction

template_message = TemplateSendMessage(
    alt_text='這是按鈕模板訊息',
    template=ButtonsTemplate(
        thumbnail_image_url='https://example.com/image.jpg',  # 可選的圖片 URL
        title='請選擇',
        text='請從以下選項中選擇：',
        actions=[
            MessageAction(label="選項1", text="你選擇了選項1"),
            MessageAction(label="選項2", text="你選擇了選項2"),
            URIAction(label="查看網站", uri="https://example.com")
        ]
    )
)
line_bot_api.reply_message(event.reply_token, template_message)
'''

Confirm Template **提供兩個按鈕，適合用於簡單的確認，如「是/否」或「接受/拒絕」選項。**

In [ ]:
'''
from linebot.models import TemplateSendMessage, ConfirmTemplate, MessageAction

template_message = TemplateSendMessage(
    alt_text='這是確認模板訊息',
    template=ConfirmTemplate(
        text='您確定要繼續嗎？',
        actions=[
            MessageAction(label="是", text="是"),
            MessageAction(label="否", text="否")
        ]
    )
)
line_bot_api.reply_message(event.reply_token, template_message)

'''

Carousel Template **可以顯示多個項目，每個項目都有標題、描述和按鈕選項，適合展示多個選擇項目。**

In [ ]:
'''
from linebot.models import TemplateSendMessage, CarouselTemplate, CarouselColumn, MessageAction, URIAction

template_message = TemplateSendMessage(
    alt_text='這是旋轉木馬模板訊息',
    template=CarouselTemplate(
        columns=[
            CarouselColumn(
                thumbnail_image_url='https://example.com/image1.jpg',
                title='項目1',
                text='這是項目1的描述',
                actions=[
                    MessageAction(label="選項1", text="你選擇了項目1的選項1"),
                    URIAction(label="查看項目1", uri="https://example.com/1")
                ]
            ),
            CarouselColumn(
                thumbnail_image_url='https://example.com/image2.jpg',
                title='項目2',
                text='這是項目2的描述',
                actions=[
                    MessageAction(label="選項2", text="你選擇了項目2的選項2"),
                    URIAction(label="查看項目2", uri="https://example.com/2")
                ]
            )
        ]
    )
)
line_bot_api.reply_message(event.reply_token, template_message)

'''

Image Carousel Template **是一種旋轉木馬模板，每張圖片都可以有一個按鈕，適合用來展示多張圖片，每張圖片下面都有相應的按鈕。**

In [ ]:
'''
from linebot.models import TemplateSendMessage, ImageCarouselTemplate, ImageCarouselColumn, URIAction, MessageAction

template_message = TemplateSendMessage(
    alt_text='這是圖片旋轉木馬模板訊息',
    template=ImageCarouselTemplate(
        columns=[
            ImageCarouselColumn(
                image_url='https://example.com/image1.jpg',
                action=MessageAction(label="點擊圖片1", text="你點擊了圖片1")
            ),
            ImageCarouselColumn(
                image_url='https://example.com/image2.jpg',
                action=URIAction(label="查看圖片2", uri="https://example.com/2")
            )
        ]
    )
)
line_bot_api.reply_message(event.reply_token, template_message)

'''

貼圖範例

In [ ]:
'''
from linebot.models import StickerSendMessage

def reply_with_sticker(event, emotion):
    if emotion == "happy":
        sticker_message = StickerSendMessage(package_id='1', sticker_id='13')  # 開心的貼圖
    elif emotion == "sad":
        sticker_message = StickerSendMessage(package_id='1', sticker_id='9')   # 傷心的貼圖
    else:
        sticker_message = StickerSendMessage(package_id='1', sticker_id='1')   # 通用的貼圖

    line_bot_api.reply_message(event.reply_token, sticker_message)

# 使用範例
reply_with_sticker(event, "happy")

'''

Emoji (可以直接加在文字中)  index(位置)

In [ ]:
'''
from linebot.models import TextSendMessage, Emoji

# 動態生成多個 Emoji
emojis_list = [
    Emoji(index=6, product_id="5ac1bfd5040ab15980c9b435", emoji_id="001"),  # 笑臉
    Emoji(index=15, product_id="5ac1bfd5040ab15980c9b435", emoji_id="002")  # 愛心
]

message = TextSendMessage(
    text="這是動態生成的多個 emoji 訊息。",
    emojis=emojis_list
)
line_bot_api.reply_message(event.reply_token, message)
'''

In [ ]:
'''
from linebot.models import TextSendMessage, Emoji

def create_emoji_message(user_text):
    # 動態生成 emoji 列表
    emojis_list = []
    index = user_text.find("特定關鍵詞") + 4  # 假設在特定關鍵詞後插入 emoji
    if index != -1:
        emojis_list.append(Emoji(index=index, product_id="5ac1bfd5040ab15980c9b435", emoji_id="001"))  # 笑臉

    # 傳送包含動態 emoji 的訊息
    message = TextSendMessage(text=user_text, emojis=emojis_list)
    return message

# 使用範例
message = create_emoji_message("Hello 特定關鍵詞!")
line_bot_api.reply_message(event.reply_token, message)

'''